In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models, applications
from tensorflow.keras.layers import Input
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
import os
from PIL import Image

def check_image_shapes(dataset_path, target_shape=(300, 300)):
    mismatched_shapes = []

    for root, dirs, files in os.walk(dataset_path):
        for breed_folder in dirs:
            breed_folder_path = os.path.join(root, breed_folder)

            for image_file in os.listdir(breed_folder_path):
                image_path = os.path.join(breed_folder_path, image_file)

                try:
                    img = Image.open(image_path)
                    width, height = img.size

                    if (width, height) != target_shape:
                        mismatched_shapes.append((image_path, (width, height)))

                except Exception as e:
                    print(f"Error processing {image_path}: {e}")

    return mismatched_shapes

# Specify the correct path using os.path.abspath
dataset_path = os.path.abspath('Cow Weight Detection/Cattle Breeds')

print(f"Checking images in path: {dataset_path}")

mismatched_shapes = check_image_shapes(dataset_path)

if not mismatched_shapes:
    print("All images have the correct shape (300x300).")
else:
    print("Mismatched image shapes:")
    for image_path, shape in mismatched_shapes:
        print(f"{image_path}: {shape}")

Checking images in path: C:\Users\Admin\Cow Weight Detection\Cow Weight Detection\Cattle Breeds
All images have the correct shape (300x300).


In [3]:
base_model = applications.VGG16(weights='imagenet', include_top=False, input_tensor=Input(shape=(300, 300, 3)))

In [4]:
# Freeze the convolutional layers
for layer in base_model.layers:
    layer.trainable = False

In [5]:
# Building model on top of the pre-trained base model
model = models.Sequential()
model.add(base_model)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(300 * 300 * 3, activation='linear'))  # Adjust output shape for regression
model.add(layers.Reshape((300, 300, 3)))  # Reshape to match target shape

In [6]:
optimizer = tf.optimizers.Adam()  # Use the updated optimizer
model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=['mae'])

In [7]:
train_data_path = 'Cattle Breeds/'
validation_data_path = 'Cattle Breeds/'

# Create an ImageDataGenerator
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Create data generators
train_generator = datagen.flow_from_directory(
    train_data_path,
    target_size=(300, 300),
    batch_size=32,
    class_mode='input'  # Regression task
)

validation_generator = datagen.flow_from_directory(
    validation_data_path,
    target_size=(300, 300),
    batch_size=32,
    class_mode='input'
)

Found 1208 images belonging to 5 classes.
Found 1208 images belonging to 5 classes.


In [8]:
model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator,
    verbose=1
)

Epoch 1/10


38/38 [==============================] - 716s 19s/step - loss: 0.1205 - mae: 0.2796 - val_loss: 0.0784 - val_mae: 0.2319
Epoch 2/10
38/38 [==============================] - 701s 19s/step - loss: 0.0719 - mae: 0.2220 - val_loss: 0.0684 - val_mae: 0.2158
Epoch 3/10
38/38 [==============================] - 702s 19s/step - loss: 0.0669 - mae: 0.2123 - val_loss: 0.0648 - val_mae: 0.2083
Epoch 4/10
38/38 [==============================] - 697s 19s/step - loss: 0.0639 - mae: 0.2062 - val_loss: 0.0618 - val_mae: 0.2024
Epoch 5/10
38/38 [==============================] - 712s 19s/step - loss: 0.0599 - mae: 0.1986 - val_loss: 0.0608 - val_mae: 0.1981
Epoch 6/10
38/38 [==============================] - 741s 20s/step - loss: 0.0589 - mae: 0.1961 - val_loss: 0.0584 - val_mae: 0.1939
Epoch 7/10
38/38 [==============================] - 700s 19s/step - loss: 0.0584 - mae: 0.1946 - val_loss: 0.0572 - val_mae: 0.1933
Epoch 8/10
38/38 [==============================] - 722s 19s/step - loss: 

In [11]:
# Save the model in HDF5 format with overwrite=True
model.save('cow_weight_model.h5', overwrite=True)